In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 198 kB 48.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7e039815ef845dbde39db40e7b8170e209dfcfe18ca414cc58a3ee5f9f5e8c4c
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_3")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [ ]:
from pyspark.sql.functions import *

In [ ]:
data = sc.textFile('raw_sales.csv')
data.take(3)

['datesold,postcode,price,propertyType,bedrooms',
 '2007-07-08 00:00:00,2600,327000,house,1',
 '2007-08-16 00:00:00,2600,790000,house,4']

In [ ]:
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)

In [ ]:
df.registerTempTable('df')
df.show(3)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|
|2007-08-16 00:00:00|    2600|790000|       house|       4|
|2007-12-05 00:00:00|    2600|825000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 3 rows



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window


windSpec = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')
df.show(3)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|
|2007-08-16 00:00:00|    2600|790000|       house|       4|
|2007-12-05 00:00:00|    2600|825000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 3 rows



In [ ]:
spark.sql('select *, row_number() over ( partition by postcode order by datesold ) as rn from df').show(3)

+-------------------+--------+------+------------+--------+---+
|           datesold|postcode| price|propertyType|bedrooms| rn|
+-------------------+--------+------+------------+--------+---+
|2007-07-02 00:00:00|    2914|800000|       house|       5|  1|
|2008-06-17 00:00:00|    2914|600000|       house|       4|  2|
|2008-08-29 00:00:00|    2914|465000|       house|       4|  3|
+-------------------+--------+------+------------+--------+---+
only showing top 3 rows



## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [ ]:
windSpec = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(-10, Window.currentRow)
df.withColumn('rn', F.avg('price').over(windSpec)).show(5)

+-------------------+--------+------+------------+--------+-----------------+
|           datesold|postcode| price|propertyType|bedrooms|               rn|
+-------------------+--------+------+------------+--------+-----------------+
|2007-07-02 00:00:00|    2914|800000|       house|       5|         800000.0|
|2008-06-17 00:00:00|    2914|600000|       house|       4|         700000.0|
|2008-08-29 00:00:00|    2914|465000|       house|       4|621666.6666666666|
|2008-09-02 00:00:00|    2914|541000|       house|       4|         601500.0|
|2008-09-05 00:00:00|    2914|395000|       house|       3|         560200.0|
+-------------------+--------+------+------------+--------+-----------------+
only showing top 5 rows



In [ ]:
windSpec = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow, 10)
df.withColumn('rn', F.avg('price').over(windSpec)).show(5)

+-------------------+--------+------+------------+--------+------------------+
|           datesold|postcode| price|propertyType|bedrooms|                rn|
+-------------------+--------+------+------------+--------+------------------+
|2007-07-02 00:00:00|    2914|800000|       house|       5| 529818.1818181818|
|2008-06-17 00:00:00|    2914|600000|       house|       4| 497090.9090909091|
|2008-08-29 00:00:00|    2914|465000|       house|       4| 485727.2727272727|
|2008-09-02 00:00:00|    2914|541000|       house|       4|486863.63636363635|
|2008-09-05 00:00:00|    2914|395000|       house|       3| 486772.7272727273|
+-------------------+--------+------+------------+--------+------------------+
only showing top 5 rows



In [ ]:
windSpec = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(-1, -1)
df.withColumn('rn', F.avg('price').over(windSpec)).show(5)

+-------------------+--------+------+------------+--------+--------+
|           datesold|postcode| price|propertyType|bedrooms|      rn|
+-------------------+--------+------+------------+--------+--------+
|2007-07-02 00:00:00|    2914|800000|       house|       5|    null|
|2008-06-17 00:00:00|    2914|600000|       house|       4|800000.0|
|2008-08-29 00:00:00|    2914|465000|       house|       4|600000.0|
|2008-09-02 00:00:00|    2914|541000|       house|       4|465000.0|
|2008-09-05 00:00:00|    2914|395000|       house|       3|541000.0|
+-------------------+--------+------+------------+--------+--------+
only showing top 5 rows



## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [ ]:
from pyspark.sql.functions import *

new_df = df.withColumn('Year', year(df['datesold']))
new_df.show()

+-------------------+--------+-------+------------+--------+----+
|           datesold|postcode|  price|propertyType|bedrooms|Year|
+-------------------+--------+-------+------------+--------+----+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|2007|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|2007|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|2007|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|2008|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|2008|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|2008|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|2008|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|2008|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|2008|
|2008-09-08 00:00:00|    2600| 740000|       house|       3|2008|
|2008-09-17 00:00:00|    2600| 720000|       house|       3|2008|
|2008-09-22 00:00:00|    2600| 690000|       house|       4|2008|
|2008-11-1

In [ ]:
new_df.groupBy('Year').agg(f.mean('price').alias('Price_average')).orderBy('Year').show()

+----+-----------------+
|Year|    Price_average|
+----+-----------------+
|2007|522377.2108843537|
|2008|493814.1627543036|
|2009|496092.0329593268|
|2010|559564.8102893891|
|2011|566715.1102265768|
|2012|552501.3692142088|
|2013|553416.2968381312|
|2014|  592653.76143905|
|2015|626101.3426535088|
|2016|635185.3116683726|
|2017| 671880.551420392|
|2018|660701.0427682737|
|2019|634184.2187725632|
+----+-----------------+



(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)

In [ ]:
df_left = df.join(new_df, on=['datesold', 'price'], how='left')
df_left.show()

+-------------------+-------+--------+------------+--------+--------+------------+--------+----+
|           datesold|  price|postcode|propertyType|bedrooms|postcode|propertyType|bedrooms|Year|
+-------------------+-------+--------+------------+--------+--------+------------+--------+----+
|2007-07-08 00:00:00| 327000|    2600|       house|       1|    2600|       house|       1|2007|
|2007-08-16 00:00:00| 790000|    2600|       house|       4|    2600|       house|       4|2007|
|2007-12-05 00:00:00| 825000|    2600|       house|       3|    2600|       house|       3|2007|
|2008-01-21 00:00:00| 315000|    2600|        unit|       1|    2600|        unit|       1|2008|
|2008-04-24 00:00:00| 292500|    2600|       house|       1|    2600|       house|       1|2008|
|2008-05-30 00:00:00| 329000|    2600|        unit|       2|    2600|        unit|       2|2008|
|2008-06-19 00:00:00| 765000|    2600|       house|       5|    2600|       house|       5|2008|
|2008-07-29 00:00:00| 927000| 

In [ ]:
df.join(df, on=['datesold', 'price'], how='left').show()

+-------------------+-------+--------+------------+--------+--------+------------+--------+
|           datesold|  price|postcode|propertyType|bedrooms|postcode|propertyType|bedrooms|
+-------------------+-------+--------+------------+--------+--------+------------+--------+
|2007-07-08 00:00:00| 327000|    2600|       house|       1|    2600|       house|       1|
|2007-08-16 00:00:00| 790000|    2600|       house|       4|    2600|       house|       4|
|2007-12-05 00:00:00| 825000|    2600|       house|       3|    2600|       house|       3|
|2008-01-21 00:00:00| 315000|    2600|        unit|       1|    2600|        unit|       1|
|2008-04-24 00:00:00| 292500|    2600|       house|       1|    2600|       house|       1|
|2008-05-30 00:00:00| 329000|    2600|        unit|       2|    2600|        unit|       2|
|2008-06-19 00:00:00| 765000|    2600|       house|       5|    2600|       house|       5|
|2008-07-29 00:00:00| 927000|    2600|       house|       4|    2600|       hous

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [ ]:
df.drop_duplicates()
df.drop_duplicates(keep='last')
pandas.dataframe.column-name.unique()
